# HW4 - Implement an Inverted Index#

### Before You Start
For this problem set, you should download INF0202-HW4.ipynb from bCourses. Create a local copy of the notebook and rename it LASTNAME_FIRSTNAME-HW4.ipynb. Then edit your renamed file directly in your browser by typing:
```
jupyter notebook <name_of_downloaded_file>
```

In [31]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from sklearn import feature_extraction

from nltk import NaiveBayesClassifier
from nltk.tokenize import word_tokenize
from itertools import chain
from sklearn.feature_extraction.text import CountVectorizer


from sklearn.model_selection import train_test_split


### Working in a group?
List the names of other students with whom you worked on this problem set:


In [3]:
# names here

### Introduction to the assignment

For this assignment, you will be creating a search engine using an inverted index. The goal is to enable the user to find all movies that match a query.

##### Use the following commands to load the dataset:

In [32]:
# load dataset
imdb = pd.read_csv("IMDB_movies_full.csv", low_memory=False, encoding = "ISO-8859-1")

In [33]:
imdb.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


First, we need to process our movie descriptions like we did in previous assignments. Then, all movies will be assigned numbers according to their position in the list: the first is movie 0, the second is number 1, and so on.

If you search for the word "in", the search engine should report that this word appears in the description of movie X and movie Y but not in movie C. A simple data structure, called an inverted index, supports such searches. An inverted index for our dataset will map each word appearing in the movie descriptions to the set of numbers of movies whose descriptions contain the word. 

We will be using a Python dictionary to do so. Each key is a word appearing in the database, and the corresponding value is the set of numbers of movies containing the word. 

### Formatting Our Data

**Task 1:** Convert movie descriptions into a list of strings. Display first 5 items.

In [34]:
description_list = list(imdb['Description'])
description_list[0:5]


['A group of intergalactic criminals are forced to work together to stop a fanatical warrior from taking control of the universe.',
 'Following clues to the origin of mankind, a team finds a structure on a distant moon, but they soon realize they are not alone.',
 'Three girls are kidnapped by a man with a diagnosed 23 distinct personalities. They must try to escape before the apparent emergence of a frightful new 24th.',
 "In a city of humanoid animals, a hustling theater impresario's attempt to save his theater with a singing competition becomes grander than he anticipates even as its finalists' find that their lives will never be the same.",
 'A secret government agency recruits some of the most dangerous incarcerated super-villains to form a defensive task force. Their first mission: save the world from the apocalypse.']

### Making Our Inverted Index

**Task 2:** Write a function `make_inverted_index(strlist)` that, given a list of strings (movie descriptions), returns a dictionary that maps each word to the set consisting of the movie numbers of movies' descriptions in which that words appears. Details on the function's inputs and outputs are in the doc string below. 

In [36]:
def make_inverted_index(strlist):
    """
    Input: a list of movies descriptions as strings
    Output: a dictionary that maps each word in any document to the set consisting of the
            movie ids (ie, the index in the strlist) for all movie descriptions containing the word.
    
    Example:
    >>> make_inverted_index(['hello world','hello','hello cat','hellolot of cats'])
    >>> {'hello': {0, 1, 2}, 'cat': {2}, 'of': {3}, 'world': {0}, 'cats': {3}, 'hellolot': {3}}
    """
    
    # find all unique words in strlist
    long_str = " ".join(strlist)
    all_words = word_tokenize(long_str.lower())
    unique_words = set(all_words)
    
    # create dictionary of sets to return 
    inverted_index = {}
    for i in unique_words:
        inverted_index[i] = set()
    print  (inverted_index)  
    # loop through strlist and add index values to each set
    for i in range(len(strlist)):

        for j in strlist[i].split():
            if j in inverted_index:
                inverted_index[j].add(i)
                    
    return inverted_index
make_inverted_index(['hello world','hello','hello cat','hellolot of cats'])

{'of': set(), 'hello': set(), 'world': set(), 'hellolot': set(), 'cat': set(), 'cats': set()}


{'of': {3},
 'hello': {0, 1, 2},
 'world': {0},
 'hellolot': {3},
 'cat': {2},
 'cats': {3}}

### Searching our Dictionary

**Task 3:** Write a function `andSearch(inverted_index, query)` which takes an inverted index and a list of words, `query` and returns the set of document numbers specifying all documents that contain *all* of the words in query. Hint: use set.intersection(), see [documentation](https://docs.python.org/2/library/sets.html).

In [37]:
def and_search(inverted_index, query):
    """
    Input: an inverted index, as created by make_inverted_index, and a list of words to query
    Output: the set of all document ids that contain _all_ of the specified words

    Example:
    >>> idx = make_inverted_index(['Johann Sebastian Bach', 'Johannes Brahms', 'Johann Strauss the Younger', 'Johann Strauss the Elder', ' Johann Christian Bach',  'Carl Philipp Emanuel Bach'])
    >>> and_search(idx, ['Johann', 'the'])
    {2, 3}
    >>> and_search(idx, ['Johann', 'Bach'])
    {0, 4}
    """
    results = set()
    
    query_list = []
    for word in query:
        if word in inverted_index:
            query_list.append(inverted_index[word])
            print(inverted_index[word])
        else:
            print('false')
    print(query_list)        
    results = set.intersection(*query_list)
    return results
test = make_inverted_index(['hello world','hello','hello cat','hellolot of cats'])

and_search(test, ['hello' ,'world', ])


{'of': set(), 'hello': set(), 'world': set(), 'hellolot': set(), 'cat': set(), 'cats': set()}
{0, 1, 2}
{0}
[{0, 1, 2}, {0}]


{0}

**What do you notice?**

So for making the inverted index I can see that it's joining the list of strings we put in as one long string (with spaces in between). Then, it's tokenizing that string so we can get each word separated. THen we put these words into a set so we can make sure there are no duplicates.

We then make a dictionary and for each word in our set, we create a key for each word with a set as its values. This is done so we can add index positions to it later. Then, for each word on the original string list, we check if its in the inverted index. If it is, then we add the index position of the string to the set for that word. At the end, we then print out the sets and the index positions in which they appear in.



For the second code, we make a set of results that will hold the instances where the query words exist in each inverted index passed in. We then make a dictionary for our query list. For each word in the query, we check if the word is in the inverted index. If it is, then we add it to the query list so we can use set.intersection on it later. 

Once we've gone through every word we use set.intersection on query_list to find the instances of each word that appear in every set (which we pass into results). We then return the results and index positions of the strings in where the words appear.